# Создание робота selenium

In [10]:
import time
import json

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from robot.helpers.selenium_management import open_link, get_wait_element, get_wait_elements, get_link_elements, get_links, start_driver, close_driver

from robot.conf import config
from robot.helpers.utils import (
    extract_emails,
    validate_instagram_url,
    POST_VALUE,
    ACCOUNT_VALUE
)




In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms"
SAMPLE_RANGE_NAME = "Class Data!A2:E"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    print("Name, Major:")
    for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      print(f"{row[0]}, {row[4]}")
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()

# Обучение модели

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from robot.ml.custom_transformers import TextCleaner, DomainBinarizer
from robot.models import AccountType

In [34]:
##################################
# Шаг 1: Предобработка текста
##################################

# Создание DataFrame
file_path = 'data/modеls/train_data/merged_table.xlsx'
df = pd.read_excel(file_path)

# Замена пустых значений
df.fillna(
    {   
        'Предсказанный тип аккаунта': 'OTHER',
        'Описание страницы': '',
        'Ссылки из описания': '', 
        'Ссылки из контактов': '',
        'Кол-во постов': 0
    }, 
    inplace=True
)

# 1 шаг: Замена имени столбца
# 2 шаг: Удаление всех строк с типом "OTHER"
df.rename(columns={"Предсказанный тип аккаунта": "Тип аккаунта"}, inplace=True)
# df = df[df["Тип аккаунта"] != "OTHER"]

# Заменяем значения столбца на 1, если не NaN, и на 0, если NaN
# 1 шаг - замена на True и False
# 2 шаг - замена на 1 и 0
df['Почта существует'] = df['Найденная почта'].notna().astype(int)

# Разделяем ссылки в строках через '\n'
df['Ссылки из описания'] = df['Ссылки из описания'].str.split('\n')
df['Ссылки из контактов'] = df['Ссылки из контактов'].str.split('\n')


##################################
# Шаг 2: Делим на X и y + разделяем данные на обучающую и тестовую выборки
##################################
X = df[[
    "Описание страницы",  # text
    "Ссылки из описания",  # text
    "Ссылки из контактов",  # text
    "Почта существует",  # 1/0
    "Кол-во постов",  # num
]]
y = df["Тип аккаунта"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

##################################
# Шаг 3: Настраиваем векторизацию и классификацию + cобираем Pipeline 
##################################

# Создание ColumnTransformer для обработки каждого столбца
preprocessor = ColumnTransformer(
    transformers=[
        (
            'desc_tfidf',  # Для текстовых значений 
            Pipeline([
                ('text_cleaner', TextCleaner()),
                ('tfidf', TfidfVectorizer(stop_words='english',  # Стоп-слова
                                          ngram_range=(1, 3),  # Использование триграмм
                                          max_features=3500,  # Увеличение числа признаков
                                          sublinear_tf=True)  # Использование логарифмического масштаба для частоты терминов
                ),
            ]),
            'Описание страницы'
            
        ),
        (
            'desc_links_binarizer',
            Pipeline([
                ('binarizer', DomainBinarizer())
                # при желании можно добавить StandardScaler() —
                # но для бинарных фич это обычно не критично
            ]),
            'Ссылки из описания'
        ),
        (
            'contact_links_binarizer',
            Pipeline([
                ('binarizer', DomainBinarizer())
            ]),
            'Ссылки из контактов'
        ),
        (
            'binary', 
            'passthrough',  # Пропускает значения без изменений
            ['Почта существует']
        ),
        (
            'num_scaler', 
            StandardScaler(),  # Для числовых значений
            ['Кол-во постов']
        ),
    ],
    remainder='drop'  # Удаляет остальные столбцы, если они есть
)


##################################
# Шаг 4: Собираем Pipeline
##################################

# Использование методов ресэмплинга
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),  # Увеличение числа примеров для редких классов с помощью методов, таких как SMOTE.
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000)),
])

# ##################################
# # Шаг 5: Обучение и оценка
# ##################################

# Обучаем
pipeline.fit(X_train, y_train)

# Предсказываем
y_pred = pipeline.predict(X_test)

# Смотрим отчёт
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

      ARTIST       0.89      0.92      0.90        36
   BEATMAKER       1.00      0.50      0.67         2
   COMMUNITY       0.78      0.95      0.86        19
       LABEL       1.00      0.60      0.75         5
      MARKET       1.00      0.67      0.80         6

    accuracy                           0.87        68
   macro avg       0.93      0.73      0.80        68
weighted avg       0.88      0.87      0.86        68



In [35]:
import joblib

##################################
# Шаг 6: Обучение модели на всех имеющихся данных
##################################

pipeline.fit(X, y)

##################################
# Шаг 7: Сохранение модели
##################################

model_path = 'data/modеls/account_type.pkl'
joblib.dump(pipeline, model_path)
print(f"Модель сохранена в файле {model_path}")


##################################
# Шаг 8: Загрузка модели
##################################

loaded_model = joblib.load(model_path)
print(f"Модель загружена из файла {model_path}")


##################################
# Шаг 9: Использование модели
##################################

new_data = pd.DataFrame({
    'Описание страницы': [
        '''Colt Blumenthal
mixedbycolt
🚀 #1 Billboard Engineer, 💿 Multi-Platinum
💻Engineered for 100+ Major Artists
🏅 SAE Institute Alumni Hall of Fame
🔊Premium Mixing & Mastering Service
linktr.ee/mixedbycolt'''
    ],
    'Ссылки из описания': [
        [
            'https://linktr.ee/mixedbycolt?fbclid=PAZXh0bgNhZW0CMTEAAaYAv9ZUzP17l1o8wVsSVjvTAzUJT9CppNIKLuP0FENyTVKEOAH3OmmBnCA_aem_FRFJBiIdaHkZ9aYHEaCLHQ',
            'https://www.instagram.com/explore/tags/1/',
            'https://www.threads.net/@mixedbycolt?xmt=AQGzUA43HPJ5PxWfBGKXf29C7EUpu6r6IVFcdtq1jbBJa8Y'
        ]
    ],
    'Ссылки из контактов': [
        ''
        # 'https://soundcloud.com/playlist/ddw31e\nhttps://youtube.com/watch?v=12345'
    ],
    'Почта существует': [0],
    'Кол-во постов': [90]
})

preds = loaded_model.predict(new_data)
preds_enum = [AccountType(s) for s in preds]  # [AccountType.ARTIST, AccountType.LABEL, ...]

print("Предсказанные классы:", preds_enum)

proba = loaded_model.predict_proba(new_data)
print("Вероятности классов:", proba)

Модель сохранена в файле data/modеls/account_type.pkl
Модель загружена из файла data/modеls/account_type.pkl
Предсказанные классы: [<AccountType.BEATMAKER: 'BEATMAKER'>]
Вероятности классов: [[0.06444823 0.85820526 0.02759148 0.03312428 0.01663076]]


In [ ]:
# Подсчет каждого типа аккаунта
print(df['Тип аккаунта'].value_counts())

In [ ]:
# Фильтруем только строки, где "Предсказанный тип аккаунта" == "ARTIST"
df_artist = df[df["Предсказанный тип аккаунта"] == "ARTIST"]

# Считаем самые популярные хэштеги среди этих строк
hashtags_count = df_artist["Хэштег, по которому найден аккаунт"].value_counts()

# Смотрим результат (топ 10, например)
print(hashtags_count.head(10))


In [ ]:
duplicates_mask = df.duplicated()
print("Число дубликатов:", duplicates_mask.sum())

### Объединение данных с обработанными таблицами

In [1]:
import os
import pandas as pd
import numpy as np

from helpers.excel import write_excel

from database.orm import async_session, get_all_accounts


In [ ]:
# Папка с файлами .ods
folder_path = 'data/modеls/tables'

# Получаем список файлов .ods в папке
files = [f for f in os.listdir(folder_path) if f.endswith('.ods')]

# Создаем пустой DataFrame для объединенных данных
combined_df = pd.DataFrame()

for file in files:
    # Загружаем данные из каждого файла
    file_path = os.path.join(folder_path, file)
    data = pd.read_excel(file_path, engine='odf')  # Используем 'odf' для .ods файлов
    
    # Очистка данных
    data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'] = data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'].replace(np.nan, 'OTHER')

    # Объединяем данные по названию столбцов
    combined_df = pd.concat([combined_df, data], ignore_index=True)

async def update_account_types(async_session, data: pd.DataFrame):
    async with async_session() as session:
        # Получаем все аккаунты из базы данных
        accounts = await get_all_accounts(async_session)

        # Создаем словарь для быстрого поиска аккаунтов по ссылке
        account_dict = {account.link: account for account in accounts}

        # Обновляем типы аккаунтов на основе данных из DataFrame
        for _, row in data.iterrows():
            link = row.get('Ссылка на аккаунт').strip()
            account_type = row.get('Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER').strip()
            # print(f'link: {link}')
            # print(f'account_type: {account_type}')

            if link in account_dict and account_type:
                account = account_dict[link]
                account.account_type = account_type
                # print(f'account: {account.link}')
                # print(f'account_type: {account.account_type}')
                session.add(account)
        # Сохраняем изменения в базе данных
        await session.commit()
        
await update_account_types(async_session, combined_df)

In [ ]:
accounts = await get_all_accounts(async_session)
write_excel(accounts, out_path='data/modеls/row_data/merged_table.xlsx')